In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../") # go to parent dir

In [3]:
import gym
from gym import spaces
import numpy as np
import sciunit
import scipy

In [4]:
from src.ldmunit.models.RWCK import RWCKModel

In [5]:
from bandit import BanditTwoArmedHighLowFixed

In [6]:
paras = dict(zip(['alpha', 'alpha_c', 'beta', 'beta_c', 'w0'], [1,1,1,1,0]))
model = RWCKModel(2, 1, paras= paras)
env = BanditTwoArmedHighLowFixed()

In [7]:
def simulate(env, model, n_trials, seed=0):
    """Simulation in a given AI Gym environment."""
    #TODO: add support for env.seed()
    assert isinstance(env, gym.Env)
    assert isinstance(model, sciunit.Model)
    assert isinstance(n_trials, int)
    
    assert model.paras != None #TODO: add assert for keys


    # reset the agent state and 
    model.reset()
    init_stimulus = env._reset()

    a = np.zeros(n_trials, dtype=int) #TODO: change dtype
    r = np.zeros(n_trials, dtype=int)
    s = np.zeros(n_trials, dtype=int)

    # add the first stimulus in the environment
    s = np.insert(s, 0, init_stimulus, axis=0)

    for i in range(n_trials):
        # compute choice probabilities
        P = model.predict(s[i])

        # action based on choice probabilities
        a[i] = model.act(P)

        # generate reward based on action
        s[i+1], r[i], done, _ = env._step(a[i])

        # update choice kernel and Q weights
        model.update(s[i], r[i], a[i], done)

    # delete the extra stimulus
    s = np.delete(s, n_trials, axis=0)

    env.close()

    obs = {
        'stimuli': s,
        'actions': a,
        'rewards': r,
    }

    return obs

In [8]:
def loglike(model, stimuli, rewards, actions):
    #TODO: add assertion for the length
    n_trials = len(stimuli)
    
    res = 0
    
    # reset the model's state
    model.reset()
    done = False
    
    for i in range(n_trials):
        # compute choice probabilities
        P = model.predict(stimuli[i])
        
        # probability of the action
        #TODO: generalize this
        p = model.loglikelihood(P, actions[i])

        # add log-likelihood
        res += np.log(p)
        # update choice kernel and Q weights
        model.update(stimuli[i], rewards[i], actions[i], done)
    
    return res

In [9]:
def fit(model, stimuli, rewards, actions, fixed):
    
    vals = list(fixed.values())

    def objective_fun(vals):
        for k, v in zip(fixed.keys(), vals):
            model.paras[k] = v
        return -loglike(model, stimuli, rewards, actions)

    opt_results = scipy.optimize.minimize(fun=objective_fun,
                                          x0=vals, 
                                          bounds=[(0,1000)]*len(vals))
    if opt_results.success:
        for k, v in zip(fixed.keys(), opt_results.x):
            model.paras[k] = v
    return opt_results

In [10]:
res = simulate(env, model, 100)

In [11]:
a = loglike(model, res['stimuli'], res['rewards'], res['actions'])
print("Log-likehood value", a)

Log-likehood value -55.97948702891962


In [12]:
print("paras before minimize", model.paras)

paras before minimize {'alpha': 1, 'alpha_c': 1, 'beta': 1, 'beta_c': 1, 'w0': 0}


In [13]:
fixed = dict(zip(['alpha', 'alpha_c', 'beta_c', 'w0'], [1,1,1,0]))
opt = fit(model, res['stimuli'], res['rewards'], res['actions'], fixed)

/anaconda3/envs/testing-ldmm/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in log
../src/ldmunit/models/RWCK.py:9: RuntimeWarning: invalid value encountered in true_divide
  return np.exp(x * beta) / np.sum(np.exp(x * beta), axis=0)


In [14]:
print("paras after minimize", model.paras)

paras after minimize {'alpha': 3.912657840187214, 'alpha_c': 0.0, 'beta': 1, 'beta_c': 1.4106348315533408, 'w0': 1e-08}
